### 목적
- 미들웨어의 핵심 개념과 내장 미들웨어에 대해 정리
- https://docs.langchain.com/oss/python/langchain/middleware

In [39]:
import os
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import AgentMiddleware

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

#### 미들웨어 개념
- 에이전트가 실행되는 동안 LLM 호출 전후, 툴 호출 전후 등 주요 지점의 동작을 가로채거나 수정할 수 있는 훅(Hook) 시스템이다.

#### AgentMiddleware와의 관계성
- AgentMiddleware는 미들웨어의 추상 부모 클래스이다.
- 모든 미들웨어는 AgentMiddleware를 상속받아서 실제 기능을 구현한다.

#### 내부 미들웨어 vs 커스텀 미들웨어
- 내부 미들웨어는 지정한 규칙(조건)을 만족하면 동작한다. 
    - LLM이 미들웨어를 선택하는 방식이 아니라 미들웨어가 관리하는 대상이 LLM이다.
- 자연어(입력)으로 자동 판단해서 키거나 끄려면 커스텀 미들웨어를 사용하여 로직을 추가해야 한다.
    - 사용자 의도를 보고 판단해 플래그 on/off

#### 내부(Built-in) 미들웨어 종류
1. SummarizationMiddleware
    - 목적: 대화가 길어졌을 때 자동 요약으로 state를 압축
    - 효과: 메모리 관리와 컨텍스트 절약
    - 핵심: 특정 길이를 넘으면 자동으로 messages를 요약
2. HumanInTheLoopMiddleware
    - 목적: 모델이 내린 결정을 사람이 승인하도록 중간 개입
    - 특징:
        - 자동화 루프 중간에 "이 응답 괜찮습니까?" 식의 검증 가능
        - 실제 프로덕션 환경의 안전장치로 활용
3. AnthropicPromptCachingMiddleware
    - 목적: Anthropic 모델에서 반복되는 긴 프롬프트를 캐싱해 비용과 지연시간을 줄임
    - 특징: 
        - 동일한 시스템/컨텍스트 프롬프트가 다시 쓰일 때 캐시에서 바로 불러옴
        - 프롬프트 길이가 큰 에이전트에서 토큰 사용량 절감
4. ModelCallLimitMiddleware
    - 목적: 모델 호출 횟수를 제한하여 무한 루프나 과도한 호출을 방지
    - 특징:
        - max_model_calls를 초과하면 자동 종료
        - 대화나 스레드 단위로 제한 설정 가능
5. ToolCallLimitMiddleware
    - 목적: 툴 호출 횟수를 전역 혹은 특정 툴별로 제한
    - 특징: 
        - max_tool_calls로 jump_to(제한 도달 시 행동) 옵션 제공
        - 모델이 같은 툴을 과도하게 반복 호출하는 현상 방지
6. ModelFallbackMiddleware
    - 목적: 주 모델 호출이 실패하거나 시간 초과될 경우 대체 모델(fallback) 자동 호출
    - 특징:
        - 여러 모델을 순차적으로 시도 가능
        - 장애 복원력 향상
7. PIIMiddleware
    - 목적: 입력/출력에서 개인정보(PII) 를 탐지하고 차단하거나 마스킹
    - 특징: 
        - 입력(before_model)과 출력(after_model), 툴 결과(after_tool) 모두 검사 가능
        - 사용자 정의(커스텀 함수/패턴)도 설정 가능
        - 민감정보 포함된 대화 처리에 필수적
8. TodoListMiddleware
    - 목적: 에이전트의 계획(plan)을 할 일 목록(todo list)으로 관리
    - 특징: 
        - 자동으로 write_todos 툴과 가이드 프롬프트를 추가
        - LLM이 "현재 진행 중인 작업 목록"을 추적하고 업데이트 가능
        - 장기적 과업 관리, 순차적 실행 제어에 용이
9. LLMToolSelectorMiddleware
    - 목적: 메인 모델 호출 전에 저비용 모델로 툴 후보를 선별해 효율 향상
    - 특징:
        - 현재 질의에 유효할 가능성이 높은 툴만 남긴다.
        - 툴이 많은 환경에서 속도 및 비용 최적화
10. ToolRetryMiddleware
    - 목적: 툴 호출 실패 시 자동 재시도
    - 특징:
        - 네트워크 에러나 일시적 오류 발생 시 일정 횟수 재시도
        - 지수 증가 간격 및 실패 시 행동 옵션 지원
        - 안정성 향상 및 일시적 장애 복원에 효과적
11. LLMToolEmulator
    - 목적: 실제 툴 대신 LLM이 툴 실행 결과를 가짜로 생성
    - 특징:
        - 특정 툴만 선택적으로 에뮬레이션 가능
        - 모델이 툴의 출력을 흉내내도록 프롬프트 설계
        - 테스트나 프로토타이핑 단계에서 빠른 시뮬레이션 가능
12. ContextEditingMiddleware
    - 목적: 컨텍스트(messages)를 편집, 정리, 절단하여 토큰 사용을 관리
    - 특징:
        - 불필요한 툴 호출 기록이나 장문 대화를 정리해 효율 개선
        - 대화 길이 제어 및 실행 속도 향상

#### 커스텀 미들웨어
- 내장 미들웨어는 고정된 규칙으로 동작한다.
    - 예시) 
        - 길이 초과 -> 요약
        - 오류 -> 재시도
- 커스텀 미들웨어는 **사용자 정의 로직**을 추가해 특정 조건에서만 동작하게 할 수 있다.


In [40]:
class CustomMiddleware(AgentMiddleware):
    def before_model(self, state, runtime):
        if "요약해줘" in state["messages"][-1]["content"]:
            runtime["use_summary"] = True

#### 미들웨어 적용 방법
- 여러 개의 미들웨어를 리스트 형식으로 지정

In [ ]:

from langchain.agents.middleware import (
    SummarizationMiddleware, 
    HumanInTheLoopMiddleware, 
    ModelCallLimitMiddleware,
    PIIMiddleware
)
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=llm,
    tools=[],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # Require approval, editing, or rejection for sending emails
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                # Auto-approve reading emails
                "read_email_tool": False,
            }
        ),
        SummarizationMiddleware(
            model="gpt-4o-mini",
            max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
            summary_prompt="Custom prompt for summarization...",  # Optional
        ),
        # 더 필요한 미들웨어는 리스트에 추가
    ]
)